#imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importando dados
fifa_df = pd.read_csv('../completo.csv')
missing = fifa_df.isnull().sum()
missing

ID                                 0
name                               0
full_name                          0
club                             253
club_logo                        253
special                            0
age                                0
league                           253
birth_date                         0
height_cm                          0
weight_kg                          0
body_type                          0
real_face                          0
flag                               0
nationality                        0
photo                              0
eur_value                          0
eur_wage                           0
eur_release_clause              1494
overall                            0
potential                          0
pac                                0
sho                                0
pas                                0
dri                                0
def                                0
phy                                0
i

In [3]:
# Tirando todos os espaços em branco antes e depois das palavras
fifa_df = fifa_df.applymap(lambda x: x.strip() if type(x) is str else x)

## 2.0 Separando variáveis categóricas das variáveis numéricas

In [4]:
numerical_variables = ['eur_value', 'eur_wage', 'eur_release_clause','age', 'height_cm', 'weight_kg', 'ID', 'special', 'overall', 'potential', 'pac', 'sho', 'pas', 'dri', 'def', 'phy', 'international_reputation', 'skill_moves', 'weak_foot', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength',
'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'rs', 'rw', 'rf', 'ram', 'rcm', 'rm', 'rdm', 'rcb', 'rb', 'rwb', 'st', 'lw', 'cf', 'cam', 'cm', 'lm', 'cdm', 'cb', 'lb', 'lwb', 'ls', 'lf', 'lam', 'lcm', 'ldm', 'lcb', 'gk']
categorical_variables = [v for v in fifa_df.columns if v not in numerical_variables]

## 3.0 Tratando variáveis Categóricas

In [5]:
fifa_df[categorical_variables].astype(str).describe()

,name,full_name,club,club_logo,league,birth_date,body_type,real_face,flag,nationality,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
count,17994,17994,17994,17994,17994,17994,17994,17994,17994,17994,...,17994,17994,17994,17994,17994,17994,17994,17994,17994,17994
unique,17022,17891,648,648,42,6010,9,2,164,164,...,2,2,2,1,1,1,1,1,1,2
top,J. Rodríguez,Nathan Smith,nan,nan,Argentinian Superliga,1992-02-29,Normal,False,https://cdn.sofifa.org/flags/14@3x.png,England,...,False,False,False,False,False,False,False,False,False,False
freq,7,3,253,253,780,225,10587,16772,1631,1631,...,14354,15931,17796,17994,17994,17994,17994,17994,17994,15973


In [6]:
# Amostra do dataset
fifa_df[categorical_variables].head()

,name,full_name,club,club_logo,league,birth_date,body_type,real_face,flag,nationality,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,Spanish Primera División,1985-02-05,C. Ronaldo,True,https://cdn.sofifa.org/flags/38@3x.png,Portugal,...,False,False,False,False,False,False,False,False,False,False
1,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,Spanish Primera División,1987-06-24,Messi,True,https://cdn.sofifa.org/flags/52@3x.png,Argentina,...,False,False,False,False,False,False,False,False,False,False
2,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,French Ligue 1,1992-02-05,Neymar,True,https://cdn.sofifa.org/flags/54@3x.png,Brazil,...,False,False,False,False,False,False,False,False,False,False
3,L. Suárez,Luis Suárez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,Spanish Primera División,1987-01-24,Normal,True,https://cdn.sofifa.org/flags/60@3x.png,Uruguay,...,False,False,False,False,False,False,False,False,False,False
4,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,German Bundesliga,1986-03-27,Normal,True,https://cdn.sofifa.org/flags/21@3x.png,Germany,...,False,False,False,False,False,False,False,False,False,True


In [7]:
fifa_df[categorical_variables].isnull().sum()

name                                0
full_name                           0
club                              253
club_logo                         253
league                            253
birth_date                          0
body_type                           0
real_face                           0
flag                                0
nationality                         0
photo                               0
work_rate_att                       0
work_rate_def                       0
preferred_foot                      0
1_on_1_rush_trait                   0
acrobatic_clearance_trait           0
argues_with_officials_trait         0
avoids_using_weaker_foot_trait      0
backs_into_player_trait             0
bicycle_kicks_trait                 0
cautious_with_crosses_trait         0
chip_shot_trait                     0
chipped_penalty_trait               0
comes_for_crosses_trait             0
corner_specialist_trait             0
diver_trait                         0
dives_into_t

In [8]:
## Jogadores que não possuem Clube, Logo do Clube ou Liga. Deletar.
fifa_df = fifa_df[fifa_df['club'].isna() & fifa_df['club_logo'].isna() & fifa_df['league'].isna()]
fifa_df[categorical_variables].isnull().sum()

name                                0
full_name                           0
club                              253
club_logo                         253
league                            253
birth_date                          0
body_type                           0
real_face                           0
flag                                0
nationality                         0
photo                               0
work_rate_att                       0
work_rate_def                       0
preferred_foot                      0
1_on_1_rush_trait                   0
acrobatic_clearance_trait           0
argues_with_officials_trait         0
avoids_using_weaker_foot_trait      0
backs_into_player_trait             0
bicycle_kicks_trait                 0
cautious_with_crosses_trait         0
chip_shot_trait                     0
chipped_penalty_trait               0
comes_for_crosses_trait             0
corner_specialist_trait             0
diver_trait                         0
dives_into_t

## Variáveis Numéricas

In [9]:
fifa_df[numerical_variables].isnull().sum()

eur_value                     0
eur_wage                      0
eur_release_clause          253
age                           0
height_cm                     0
weight_kg                     0
ID                            0
special                       0
overall                       0
potential                     0
pac                           0
sho                           0
pas                           0
dri                           0
def                           0
phy                           0
international_reputation      0
skill_moves                   0
weak_foot                     0
crossing                      0
finishing                     0
heading_accuracy              0
short_passing                 0
volleys                       0
dribbling                     0
curve                         0
free_kick_accuracy            0
long_passing                  0
ball_control                  0
acceleration                  0
                           ... 
gk_kicki

In [13]:
# Tabela de correlação par a par (Coeficiente de Pearson)
fifa_df[numerical_variables].corr()

,eur_value,eur_wage,eur_release_clause,age,height_cm,weight_kg,ID,special,overall,potential,...,cb,lb,lwb,ls,lf,lam,lcm,ldm,lcb,gk
eur_value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eur_wage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eur_release_clause,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,NaN,NaN,NaN,1.000000,0.136961,0.257949,-0.701148,0.149971,0.363787,-0.167340,...,0.275972,0.225051,0.228192,0.267554,0.209331,0.199075,0.261268,0.283147,0.275972,0.126434
height_cm,NaN,NaN,NaN,0.136961,1.000000,0.767354,-0.176765,-0.348245,0.083272,0.011514,...,0.239418,0.006182,-0.078198,-0.053704,-0.192251,-0.222780,-0.171156,0.055037,0.239418,0.023843
weight_kg,NaN,NaN,NaN,0.257949,0.767354,1.000000,-0.204564,-0.310409,0.113756,-0.012138,...,0.275277,0.039604,-0.034274,-0.016917,-0.156839,-0.192803,-0.128393,0.102168,0.275277,0.029542
ID,NaN,NaN,NaN,-0.701148,-0.176765,-0.204564,1.000000,-0.280501,-0.505958,-0.159499,...,-0.160565,-0.168821,-0.223116,-0.480703,-0.434411,-0.421417,-0.435818,-0.255666,-0.160565,-0.250888
special,NaN,NaN,NaN,0.149971,-0.348245,-0.310409,-0.280501,1.000000,0.470721,0.365902,...,0.286249,0.520684,0.658573,0.806614,0.844166,0.872880,0.944608,0.604000,0.286249,0.397723
overall,NaN,NaN,NaN,0.363787,0.083272,0.113756,-0.505958,0.470721,1.000000,0.794415,...,0.335257,0.452360,0.533608,0.633212,0.620916,0.640222,0.729948,0.542102,0.335257,1.000000
potential,NaN,NaN,NaN,-0.167340,0.011514,-0.012138,-0.159499,0.365902,0.794415,1.000000,...,0.165404,0.281756,0.357977,0.506579,0.513973,0.533881,0.574423,0.356920,0.165404,0.851014


In [16]:
 correlation_thresold = 0.7
# Identificando colunas altamente relacionadas
correlation_matrix = fifa_df[numerical_variables].corr().abs()
# Seleciona o triângulo superior da matriz de correlação
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
# Seleciona colunas a serem removidas
to_drop = [column for column in upper.columns if any(upper[column] > correlation_thresold)]
to_drop

['weight_kg',
 'ID',
 'potential',
 'dri',
 'skill_moves',
 'crossing',
 'finishing',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'composure',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'rs',
 'rw',
 'rf',
 'ram',
 'rcm',
 'rm',
 'rdm',
 'rcb',
 'rb',
 'rwb',
 'st',
 'lw',
 'cf',
 'cam',
 'cm',
 'lm',
 'cdm',
 'cb',
 'lb',
 'lwb',
 'ls',
 'lf',
 'lam',
 'lcm',
 'ldm',
 'lcb',
 'gk']

In [17]:
# Removendo colunas com alta correlação e revisando as colunas disponíveis
numerical_df.drop(numerical_df[to_drop], axis='columns', inplace=True)
numerical_df.isnull().sum()

NameError: name 'numerical_df' is not defined

# 4.2 Normalizando os dados

In [10]:
fifa_df[numerical_variables].head()

,eur_value,eur_wage,eur_release_clause,age,height_cm,weight_kg,ID,special,overall,potential,...,cb,lb,lwb,ls,lf,lam,lcm,ldm,lcb,gk
163,0.0,0.0,NaN,25,179.0,67.0,188152,1961,83,86,...,49.0,61.0,64.0,74.0,80.0,81.0,77.0,61.0,49.0,NaN
168,0.0,0.0,NaN,28,175.0,69.0,184826,2174,83,83,...,75.0,78.0,79.0,74.0,78.0,80.0,82.0,81.0,75.0,NaN
271,0.0,0.0,NaN,28,186.0,73.0,177413,2143,82,82,...,76.0,76.0,77.0,77.0,78.0,79.0,80.0,79.0,76.0,NaN
480,0.0,0.0,NaN,30,184.0,74.0,176733,1841,80,80,...,47.0,50.0,53.0,78.0,76.0,73.0,66.0,51.0,47.0,NaN
494,0.0,0.0,NaN,29,186.0,86.0,169195,2069,80,80,...,68.0,71.0,73.0,74.0,78.0,79.0,79.0,74.0,68.0,NaN


In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fifa_df[numerical_variables] = scaler.fit_transform(fifa_df[numerical_variables])
fifa_df[numerical_variables].head()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# 5. Codificando colunas categóricas

5.1 Codificando colunas simples

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import string

def remove_special_chars(x, exceptions = ['|']):
    x = x.replace(' ', '').lower()
    for char in string.punctuation:
        if char not in exceptions:
            x = x.replace(char, '')
    return x
string.punctuation

In [ ]:
fifa_df[categorical_variables].head()

Club, name, full name, club_logo

In [ ]:
all_clubs = fifa_df.club
acc_frequency = all_clubs.value_counts().cumsum() / len(all_clubs) * 100
x = [i for i in range(0,100, 5)]
y = [len(acc_frequency[acc_frequency < i]) for i in x]
frequencies = pd.Series(index=x, data=y)
frequencies.plot(kind='bar')

In [ ]:
##Eliminando os clubs menos frequentes - serão utilizados os que tem frequencia acumulada somada, em ordem, ate !!!!!!!!VERIFICAR
selected_clubs = list(acc_frequency[acc_frequency < 70].index)
len(selected_clubs), len(set(fifa_df.club))

In [ ]:
# substituindo os diretores irrelevantes por um diretor padrão
fifa_df.club = fifa_df.club.apply(lambda x: x if x in selected_clubs else 'other')

In [ ]:
variables = ['name', 'full name']
fifa_df = pd.get_dummies(fifa_df, columns=variables)
#movies_df.drop(columns=variables, inplace=True)
fifa_df.head()